# 1. CellTag_extract_region
input1> 190713_raw_CellTag_retarget.txt  
input2> 190713_CellTag_retarget.csv

> - POS(시작)와 PNEXT(끝)를 본다.  
> - POS값이 PNEXT보다 크면 forward sequenced이고, PNEXT값이 POS보다 크면 reverse sequence이다.  
> - POS값과 PNEXT값을 비교해서 더 큰 수를 선택한 후에 큰 수가 2520 미만이거나, 작은 수가 2536 초과이면 제외한다. (여집합 개념)    

output> 190713_CellTag_2435_2621.txt

In [14]:
import pandas as pd
# low_memory=False: error 떠서 지정해줌.
data = pd.read_csv("190713_raw_CellTag_retarget.txt", sep="\t", low_memory=False)
data.head()

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL,NM,MD,AS,XS,XA,XAA
0,D00236:931:CCKH7ANXX:2:1102:10714:34145,99,CellTag,2332,60,86M2D9M3D6M,=,2632,326,AAATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAA...,ABBCBGEGCEGGGGGGGFGGGGGGGGGGGGFGGCGGGGGFEGGGFG...,NM:i:5,MD:Z:86^AA9^TTC6,AS:i:87,XS:i:58,NaN,NaN
1,D00235:658:CDMHFANXX:3:2209:5454:88862,163,CellTag,2333,60,91M2D10M,=,2377,147,AATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAAT...,BBCB@GGGGGGEGGGGGGGFCFGGGGGGGGGGGGGGGGGGGGGGGG...,NM:i:2,MD:Z:91^TT10,AS:i:96,XS:i:64,NaN,NaN
2,D00236:930:CDNV0ANXX:1:2304:19650:31792,163,CellTag,2333,60,91M2D10M,=,2377,147,AATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAAT...,BBBCBGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,NM:i:2,MD:Z:91^TT10,AS:i:96,XS:i:64,NaN,NaN
3,D00236:931:CCKH7ANXX:2:1201:3534:7914,163,CellTag,2333,60,91M2D10M,=,2377,147,AATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAAT...,CCCCCGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,NM:i:2,MD:Z:91^TT10,AS:i:96,XS:i:64,NaN,NaN
4,D00236:931:CCKH7ANXX:2:1102:9574:66147,99,CellTag,2333,60,91M2D10M,=,2429,193,AATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAAT...,ABBBBGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGFGGGGGGGG...,NM:i:3,MD:Z:58G32^TT10,AS:i:91,XS:i:69,NaN,NaN


In [10]:
data.columns

Index(['QNAME', 'FLAG', 'RNAME', 'POS', 'MAPQ', 'CIGAR', 'RNEXT', 'PNEXT',
       'TLEN', 'SEQ', 'QUAL', 'NM', 'MD', 'AS', 'XS', 'XA', 'XAA'],
      dtype='object')

In [11]:
data.columns[11]

'NM'

In [12]:
data_cut = data.iloc[:, :11]  # QUAL column 까지만 자르기
data_cut.head()  # QUAL 까지만 나오는지 확인
data_cut.to_csv("190713_CellTag_retarget.csv", index=None)  # 저장

>11번째 column인 NM부터 MD, AS, XS... 은 필요 없는 column이므로 자른다.
>[row, column] 에서 11번째 미만 column (다시 말해, 10번째 column 까지 = QUAL) 까지 자른다.
>data_cut.head() 로 QUAL column 까지만 나오는 것을 확인 한 후 저장한다.

### input2> 190713_CellTag_retarget.csv
data_cut을 input 으로 다시 시작  

In [32]:
data_cut_length = len(data_cut)
data_cut_length

1315262

> input으로 넣어준  190713_CellTag_retarget.csv 파일의 길이가 어느정도인지 확인한다.

In [17]:
target_lines = []
line_num = 0
sample_line = data_cut.iloc[line_num,:]
sample_line

QNAME              D00236:931:CCKH7ANXX:2:1102:10714:34145
FLAG                                                    99
RNAME                                              CellTag
POS                                                   2332
MAPQ                                                    60
CIGAR                                          86M2D9M3D6M
RNEXT                                                    =
PNEXT                                                 2632
TLEN                                                   326
SEQ      AAATGAATGCAATTGTTGTTGTTAACTTGTTTATTGCAGCTTATAA...
QUAL     ABBCBGEGCEGGGGGGGFGGGGGGGGGGGGFGGCGGGGGFEGGGFG...
Name: 0, dtype: object

>target_lines 이라는 빈 list를 만들어준 다음, 데이터의 범위를 지정해주는 .iloc 를 사용한다.  
>우선 첫번째 줄 하나를 타겟해 돌려서 확인한 후 전체에 대해서 다시 실행한다.  
>line_num = 0 이라고 지정해 첫번째 줄만 확인할 것이다.  
>[행,열] 에서 행은 첫번째 행만, 열은 전체를 보여달라.

In [18]:
sample_line["POS"]

2332

In [19]:
sample_line["PNEXT"]

2632

In [20]:
if sample_line["PNEXT"] >= sample_line["POS"]:
    big_num = sample_line["PNEXT"]
    small_num = sample_line["POS"]
else:
    big_num = sample_line["POS"]
    small_num = sample_line["PNEXT"]

>만약 PNEXT가 POS보다 크거나 같으면, 큰 값은 PNEXT, 작은 값은 POS이다.  
>그렇지 않으면 큰 값은 POS, 작은 값은 PNEXT이다.

In [21]:
small_num

2332

In [22]:
big_num

2632

In [28]:
is_target = True
if big_num < 2435:
    is_target = False
if small_num > 2621:
    is_target = False
is_target

True

>default로 is_target이 true라고 지정한 후,   
>만약 POS와 PNEXT 중 더 큰 값인 big_num가 우리가 타켓한 범위의 가장 작은 값인 2435보다 작으면 is_target은 False  
>만약 POS와 PNEXT 중 더 작은 값인 small_num가 우리가 타겟한 범위의 가장 큰 값인 2621보다 크면 is_target은 False  
>우리가 예시로 넣어준 small_num = 2332, big_num = 2632는 is_target을 만족하기 때문에 True로 나올 것이다.  
>둘다 만족하지 않아야 True

In [29]:
if is_target:
    target_lines.append(line_num)

> is_target이 True인 값을 아까 만들어 놓은 빈 리스트인 target_lines에 넣어라. 

### 위에서는 하나에 대해 돌려봤다면, 지금부터는 파일 전체에 대해서 실행해 볼 것이다.

In [30]:
def filter_line(line):
    if line["PNEXT"] >= line["POS"]:
        big_num = line["PNEXT"]
        small_num = line["POS"]
    else:
        big_num = line["POS"]
        small_num = line["PNEXT"]
    is_target = True
    if big_num < 2435:
        is_target = False
    if small_num > 2621:
        is_target = False
    return is_target

> 자주 쓰이는 코드는 함수로 만들어 놓는 것이 좋다.  
> def - return 을 사용한다. (def 함수이름 지정, return 반환하고자 하는 것)

#### 진행상황을 확인할 수 없음.
>원래는 아래와 같이 하면 되지만, 여러 줄을 돌리기 때문에 진행상황을 확인하면 좋음.  
>count로 진행상황을 확인한다. (꿀팁!)   

```
target_lines = []
for line_num in range(data_cut_length):        
    line = data.iloc[line_num,:]
    is_target = filter_line(line)
    if is_target:
        target_lines.append(line_num)
```

In [33]:
target_lines = []
count = 0
for line_num in range(data_cut_length):
    count += 1
    if count % 10000 == 0:
        print(count)
        
    line = data_cut.iloc[line_num,:]
    is_target = filter_line(line)
    if is_target:
        target_lines.append(line_num)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000


#### count를 넣어서 완성 시킨 코드. (진행상황 확인 가능)
>10000줄을 읽으면 count가 프린트 된다. 진행상황이 이렇게 나와서 좋다!

In [53]:
len(target_lines)

1164378

> target_lines: 조건을 만족하는 것이 몇 개인지 알려줌.

In [55]:
target_data = data_cut.iloc[target_lines, :]
target_data.to_csv("190713_CellTag_2435_2621.txt", index=None)

> 행으로 target_lines라는 리스트에 들어가 있는 1164378줄을 모두 넣고, column은 data_cut에 있는 모든 열을 넣는다.  
> .to_csv로 저장한다. index는 각 행 앞에 0,1,2... 나오도록 하는 것인데 필요 없는 경우 None으로 지정한다.